### Function Calling and API Calls

A lot of projects with Langchain are just scripts executed on a local computer. In the "real world", production ready
services run in some kind of standardized wrapper, like an API inside a Docker container. This is an approach how you might
interact with a REST-API with an LLM Chain (which might later also be used inside an API withh a single POST endpoint)

In [28]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [29]:
import requests

def get_todos(completed=None):
    params = {'completed': completed} if completed is not None else None
    response = requests.get('https://fastapilangchain-1-w0858112.deta.app/todos', params=params)
    return response.json()

def create_todo(todo):
    response = requests.post('https://fastapilangchain-1-w0858112.deta.app/todos', json=todo)
    return response.json()


In [30]:
# Store your functions in a dictionary
api_functions = {
    "get_todos": get_todos,
    "create_todo": create_todo
}

In [31]:
functions = [
    {
        "name": "get_todos",
        "description": "Get a list of todos, optionally filtered by their completion status",
        "parameters": {
            "type": "object",
            "properties": {
                "completed": {
                    "type": "boolean",
                    "description": "Whether to only return completed todos",
                },
            },
            "required": [],
        },
    },
    {
        "name": "create_todo",
        "description": "Create a new todo",
        "parameters": {
            "type": "object",
            "properties": {
                "todo": {
                    "type": "object",
                    "description": "The new todo to be created",
                    "properties": {
                        "id": {
                            "type": "integer",
                            "description": "The id of the todo",
                        },
                        "task": {
                            "type": "string",
                            "description": "The task of the todo",
                        },
                        "is_completed": {
                            "type": "boolean",
                            "description": "Whether the task is completed",
                            "default": False
                        },
                    },
                    "required": ["task"],
                },
            },
            "required": ["todo"],
        },
    }
]

In [32]:
query="I want to walk my dog in the afternoon"

response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": query}],
        functions=functions,
    )
message = response["choices"][0]["message"]

message

<OpenAIObject at 0x1f6aa268a70> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"todo\": {\n    \"id\": 1,\n    \"task\": \"Walk the dog\",\n    \"is_completed\": false\n  }\n}",
    "name": "create_todo"
  },
  "role": "assistant"
}

In [41]:
import json

if message.get("function_call"):
    function_name = message["function_call"]["name"]
    function_args_json = message["function_call"].get("arguments", {})
    function_args = json.loads(function_args_json)

    api_function = api_functions.get(function_name)

    if api_function:
        result = str(api_function(**function_args)) 
    else:
        print(f"Function {api_function} not found")

result

"{'id': 4, 'task': 'Walk the dog', 'is_completed': False}"

In [42]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": query},
        message,
        {
            "role": "function",
            "name": function_name,
            "content": result,
        },
    ],
)

second_response

<OpenAIObject chat.completion id=chatcmpl-7S9VUCFhUpQB6cXiMbtebZYmG1Yjz at 0x1f6aa21e1b0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Great! I have created a task for you to walk your dog in the afternoon. Remember to choose a safe and comfortable route for your dog's walk. Enjoy your time together!",
        "role": "assistant"
      }
    }
  ],
  "created": 1686944084,
  "id": "chatcmpl-7S9VUCFhUpQB6cXiMbtebZYmG1Yjz",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 36,
    "prompt_tokens": 82,
    "total_tokens": 118
  }
}